The first step is to import the Notebook loader code, which enables the import of notebooks as if they were python code.

In [ ]:
import daipy.nb_loader

Next, let's import numpy and matplotlib, because we will most certainly use them.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

Let's import the Camera class from its notebook, which simplifies getting images from the USB Camera

In [ ]:
from cam import Camera
cam = Camera()

We should make sure that it's working... Let's capture a frame and display it!

In [ ]:
rgb = cam.get_rgb()
plt.imshow(rgb)
plt.show()

Let's import the DPU module and use it to load the bitstream and the DNN model!

In [ ]:
from pynq_dpu import DpuOverlay
overlay = DpuOverlay("dpu.bit")
overlay.load_model("dpu_tf_inceptionv1.xmodel")

We should take a look at the input and output tensors and make sure it lines up with what we expect.

In [ ]:
dpu = overlay.runner

inputTensors = dpu.get_input_tensors()
print("Input Tensors: " + str(inputTensors))
outputTensors = dpu.get_output_tensors()
print("Output Tensors: " + str(outputTensors))

Let's also take a look at their shapes

In [ ]:
shapeIn = tuple(inputTensors[0].dims)
print("Input Shape: " + str(shapeIn))
shapeOut = tuple(outputTensors[0].dims)
print("Output Shape: " + str(shapeOut))

We'll create an array for the softmax outputs and take a look at its shape

In [ ]:
outputSize = int(outputTensors[0].get_data_size() / shapeIn[0])
softmax = np.empty(outputSize)
print("Softmax output shape: " + str(softmax.shape))

Create the input/output buffers

In [ ]:
output_data = [np.empty(shapeOut, dtype=np.float32, order="C")]
input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
image = input_data[0]

Create a function for easy inferencing

In [ ]:
import daipy.preproc as pre
import daipy.postproc as post

In [ ]:
def run(img, display=False):
    preprocessed = pre.process(img)
    image[0,...] = preprocessed.reshape(shapeIn[1:])
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    temp = [j.reshape(1, outputSize) for j in output_data]
    softmax = post.calculate_softmax(temp[0][0])
    if display:
        _, ax = plt.subplots(1)
        _ = ax.imshow(img)
        print("Classification: {}".format(post.predict_label(softmax)))

Now we can run the inferencing for 10 camera frames

In [ ]:
for i in range(0, 10):
    run(cam.get_rgb(), display=True)

Finally, we should clean up

In [ ]:
cam.close()